In [2]:
import numpy as np
import roboticstoolbox as rtb
import spatialmath as sm
from swift import Swift
import math
import spatialgeometry as sg
from typing import Tuple

In [3]:
panda = rtb.models.Panda()
print(panda)

# Use the default start and end links
print(panda.fkine(panda.qr))

# Use string start and end links
print(panda.fkine(panda.qr, start="panda_link0", end="panda_link5"))

# Use reference start and end links
print(panda.fkine(panda.qr, start=panda.links[3], end=panda.grippers[0]))

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │ SE3()                                          │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

In [4]:
env = Swift()
# env.launch(realtime=True, browser="notebook")
env.launch(realtime=True)
env.add(panda)



0

In [5]:
# end-effector axes
ee_axes = sg.Axes(0.1)

# goal axes
goal_axes = sg.Axes(0.1)

# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes) 
env.step(0)

In [11]:
# Change the robot configuration to the ready position
panda.q = panda.qr

# Step the sim to view the robot in this configuration
env.step(0)

# A variable to specify when to break the loop
arrived = False

# Specify the gain for the p_servo method
gain = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# Specify our timestep
dt = 0.05
Tep = 0

def side_eject_direction(block_num, Tep):
    direction = "undefined"
    height = 0.015*block_num
    length_block_side = 0.025
    x_default = 0.25
    y_default = 0.25
    
    if block_num %3 == 2:
        print("Side Ejection Method, Wrong Block num")
    elif block_num %6 <= 3:
        print("left")
        direction = 1
    else:
        print("right")
        direction = 2
    
    if direction == 1:
        if block_num % 6 == 1:
            x = x_default - length_block_side
            Tep = sm.SE3.Trans(x, 0.25, height)* sm.SE3.RPY(-np.pi/2,np.pi/2, 0)
        elif block_num % 6 == 3:
            x = x_default + length_block_side
            Tep = sm.SE3.Trans(x, 0.25, height)* sm.SE3.RPY(np.pi/2,np.pi/2, 0)
    elif direction == 2:
        if block_num % 6 == 4:
            y = y_default - length_block_side
            Tep = sm.SE3.Trans(0.25, y, height)* sm.SE3.RPY(-np.pi/2,np.pi/2, 0)
        elif block_num % 6 == 3:
            y = y_default + length_block_side
            Tep = sm.SE3.Trans(0.25, y, height)* sm.SE3.RPY(np.pi/2,np.pi/2, 0)

    Tep = Tep.A
    # Set the goal axes to Tep
    goal_axes.T = Tep
    return goal_axes, Tep

# Run the simulation until the robot arrives at the goal
def target_move(Tep, arrived):
    while not arrived:

        # Work out the base frame manipulator Jacobian using the current robot configuration
        J = panda.jacob0(panda.q)

        # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
        Te = panda.fkine(panda.q).A

        # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
        # use the pseudoinverse (the pinv method)
        J_pinv = np.linalg.pinv(J)

        # Calculate the required end-effector velocity and whether the robot has arrived
        ev, arrived = rtb.p_servo(Te, Tep, gain=gain, threshold=0.001, method='angle-axis')

        # Calculate the required joint velocities and apply to the robot
        panda.qd = J_pinv @ ev

        # Update the ee axes
        ee_axes.T = Te

        # Step the simulator by dt seconds
        env.step(dt)
    print("ARRived")
    arrived = False
    return arrived

side_eject_direction(25, Tep)
target_move(Tep, arrived)


: 

: 

In [318]:

    
# Set the goal axes to Tep
goal_axes.T = sm.SE3.Trans(0.25, 0.1, 0.25) * sm.SE3.RPY(-np.pi/2, np.pi/2,0)

# Make the target move, set its velocity
# goal_axes.v = np.array([0.0, 0.02, 0.02, 0.0, 0.0, 0.0])
# goal_axes.v = np.zeros(6)

gain = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
# Run the simulation for 300 steps
# As the target never stops moving, the robot will never arrive at the goal,
# but it will continuously track towards it
while not arrived:


    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda.jacob0(panda.q)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda.fkine(panda.q).A

    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_pinv = np.linalg.pinv(J)

    # Calculate the required end-effector velocity
    # Here we feed in the goal_axes.T, which is an SE3, as our goal pose
    ev, arrived = rtb.p_servo(Te, goal_axes.T, gain=gain, threshold=0.001, method='angle-axis')

    # Calculate the required joint velocities and apply to the robot
    panda.qd = J_pinv @ ev

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)